In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-11-14 12:10:06--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  82.7MB/s    in 0.3s    

2023-11-14 12:10:06 (82.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
# add your code here - consider creating a new cell for each section of code
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
data=df_ratings.merge(df_books,on='isbn',how="left")

# data=data.drop_duplicates(['user', 'title'])

data.head()
data.shape

(1149780, 5)

In [7]:
data.query(' title=="The Queen of the Damned (Vampire Chronicles (Paperback))" ')['user'].value_counts

<bound method IndexOpsMixin.value_counts of 2049       277478
14147        2033
20332        3827
24157        5609
24535        5772
            ...  
1138818    274061
1140813    274308
1144023    275020
1145743    275673
1148042    276094
Name: user, Length: 274, dtype: int32>

In [8]:
# add your code here - consider creating a new cell for each section of code


print(data.shape)
print(list(data.columns))
# plt.rc("font", size=15)

# data.title.value_counts(sort=False).plot(kind='bar')
# plt.title('title\n')
# plt.xlabel('title')
# plt.ylabel('rating')
# # plt.savefig('system1.png', bbox_inches='tight')
# plt.show()

# data.rating.hist(bins=[0, 1,2,3,4,5,6,7,8,9,10])
# plt.title('rating\n')
# plt.xlabel('rating')
# plt.ylabel('Count')
# plt.savefig('system2.png', bbox_inches='tight')
# plt.show()

(1149780, 5)
['user', 'isbn', 'rating', 'title', 'author']


In [9]:
df=data


df2=df.value_counts(subset = ['user'])
df2 = df2.reset_index()
df2.columns = ['user','N_user']




ratings=data.merge(df2,on='user',how='left')



df3=df.value_counts(subset = ['title'])
df3 = df3.reset_index()
df3.columns = ['title','N_title']




ratings=ratings.merge(df3,on='title',how='left')





ratings.head()
# data.iloc(int(ind))

ratings2=ratings.query(' N_user>=200 & N_title>=100')
# ratings2=ratings.query('N_title>=100')
ratings2.head()

print(df.shape,df2.shape,df3.shape,ratings.shape,ratings2.shape)

(1149780, 5) (105283, 2) (241090, 2) (1149780, 7) (68365, 7)


In [10]:
len(ratings2['title'].unique())

914

In [11]:
ratings3=ratings2[['user','title','rating']]
# ratings3=ratings2[['user','title','rating','author']]
ratings3.shape

(68365, 3)

In [12]:
ratings3.head()

dff= ratings3.drop_duplicates(['user', 'title'])
print(ratings3.shape,dff.shape,len(ratings3['title'].unique()))

(68365, 3) (66286, 3) 914


In [13]:
from scipy.sparse import csr_matrix


dff_pivot = dff.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
dff_matrix = csr_matrix(dff_pivot.values)

In [14]:

dff_pivot.shape



(914, 893)

In [15]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(n_neighbors=8,metric = 'cosine', algorithm='brute')
model_knn.fit(dff_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=8)

In [16]:


dff_pivot



user,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
query_index = np.random.choice(dff_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(dff_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

27


In [18]:
for i in range(0, len(distances.flatten())):
   if i == 0:
      print('Recommendations for {0}:\n'.format(dff_pivot.index[query_index]))
   else:
      print('{0}: {1}, with distance of {2}:'.format(i, dff_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for A Separate Peace:

1: Rebecca, with distance of 0.8207268714904785:
2: The Reef, with distance of 0.8215981721878052:
3: Foucault's Pendulum, with distance of 0.8253371119499207:
4: The Secret Garden, with distance of 0.8270288705825806:
5: To Kill a Mockingbird, with distance of 0.8299840092658997:


In [19]:
"The Queen of the Damned (Vampire Chronicles (Paperback))" in dff_pivot.index

True

In [20]:
# function to return recommended books - this will be tested
def get_recommends(book):

  if book in dff_pivot.index:
     cont=0
     for i in range(len(dff_pivot.index)):
          cont=cont+1
          if dff_pivot.index[i]== book:

             ind=i

             break
     cont=ind

  query_index = cont
  # print(query_index)
  distances, indices = model_knn.kneighbors(dff_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)
  vv=[]
  recommended_books=[]
  for i in range(0, len(distances.flatten())):
   l=len(distances.flatten())-i
   if i == 0:
      # print('Recommendations for {0}:\n'.format(dff_pivot.index[query_index]))
      dff_pivot.index[query_index]
   else:
      # print('{0}: {1}, with distance of {2}:'.format(i, dff_pivot.index[indices.flatten()[i]], distances.flatten()[i]))
      vvv=[dff_pivot.index[indices.flatten()[l]], distances.flatten()[l]]
      vv.append(vvv)
  recommended_books.append(dff_pivot.index[query_index])
  recommended_books.append(vv)
  return recommended_books

In [21]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['The Witching Hour (Lives of the Mayfair Witches)', 0.74359274],
  ['Interview with the Vampire', 0.7383682],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]

In [22]:
# [
#   'The Queen of the Damned (Vampire Chronicles (Paperback))',
#   [
#     ['Catch 22', 0.793983519077301],
#     ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
#     ['Interview with the Vampire', 0.7345068454742432],
#     ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
#     ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
#   ]
# ]

In [23]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.77085835],
  ['I Know This Much Is True', 0.7518421],
  ['The Lovely Bones: A Novel', 0.7234864],
  ['Blue Diary', 0.71828747]]]

In [24]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['I Know This Much Is True', 0.7518421], ['The Lovely Bones: A Novel', 0.7234864], ['Blue Diary', 0.71828747]]]
You passed the challenge! 🎉🎉🎉🎉🎉
